In [7]:
# IPython magig  tools
%load_ext autoreload
%autoreload 2


import sys
sys.path.append('../src/')

import os
from os import PathLike
from pathlib import Path

import data_io

from aind_vr_foraging_analysis.utils.parsing import parse, data_access
import aind_vr_foraging_analysis.utils.plotting as plotting

# Plotting libraries
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

import seaborn as sns
import pandas as pd
import numpy as np
from matplotlib.patches import Rectangle

def format_func(value, tick_number):
    return f"{value:.0f}"


sns.set_context('talk')

import warnings
pd.options.mode.chained_assignment = None  # Ignore SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

from typing import TYPE_CHECKING, Annotated, Any, Dict, List, Literal, Optional, Self, TextIO, Tuple, TypeAlias, Union

import yaml

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
case DeviceYmlByRegister0(register0_glob_pattern=glob_pattern):
    # If we are allowed to infer the WhoAmI, we try to find it
    _reg_0_hint: List[os.PathLike] = []
    for pattern in glob_pattern:
        _reg_0_hint.extend(Path(params.path).glob(pattern))
    if len(_reg_0_hint) == 0:
        raise FileNotFoundError(
            "File corresponding to WhoAmI register not found given the provided glob patterns."
        )
    device_hint = int(
        harp.read(_reg_0_hint[0]).values[0][0]
    )  # We read the first line of the file to get the WhoAmI value
    _yml_stream = io.TextIOWrapper(fetch_yml_from_who_am_i(device_hint))

In [18]:
def fetch_who_am_i_list(
    url: str = "https://raw.githubusercontent.com/harp-tech/whoami/main/whoami.yml",
) -> Dict[int, Any]:
    """Fetch and parse the Harp WhoAmI registry.

    Downloads and parses the WhoAmI registry YAML file from GitHub.
    Results are cached for efficiency.

    Args:
        url: URL to the WhoAmI registry YAML file.

    Returns:
        Dict[int, Any]: Dictionary mapping WhoAmI identifiers to device information.
    """
    response = requests.get(url, allow_redirects=True, timeout=5)
    content = response.content.decode("utf-8")
    content = yaml.safe_load(content)
    devices = content["devices"]
    return devices

def fetch_yml_from_who_am_i(who_am_i: int, release: str = "main") -> io.BytesIO:
    """Fetch a device YAML file based on its WhoAmI identifier.

    Looks up the device in the WhoAmI registry and downloads its YAML file.

    Args:
        who_am_i: WhoAmI identifier of the device.
        release: Git branch or tag to use for fetching the YAML file.

    Returns:
        io.BytesIO: Memory buffer containing the device YAML content.

    Raises:
        KeyError: If the WhoAmI identifier is not found in the registry.
        ValueError: If required repository information is missing or YAML file cannot be found.
    """
    try:
        device = fetch_who_am_i_list()[who_am_i]
    except KeyError as e:
        raise KeyError(f"WhoAmI {who_am_i} not found in whoami.yml") from e

    repository_url = device.get("repositoryUrl", None)

    if repository_url is None:
        raise ValueError("Device's repositoryUrl not found in whoami.yml")

    _repo_hint_paths = [
        "{repository_url}/{release}/device.yml",
        "{repository_url}/{release}/software/bonsai/device.yml",
    ]

    yml = None
    for hint in _repo_hint_paths:
        url = hint.format(repository_url=repository_url, release=release)
        if "github.com" in url:
            url = url.replace("github.com", "raw.githubusercontent.com")
        response = requests.get(url, allow_redirects=True, timeout=5)
        if response.status_code == 200:
            yml = io.BytesIO(response.content)
            print(yml)
            return yml

    raise ValueError("device.yml not found in any repository")


In [22]:
file = io.TextIOWrapper(fetch_yml_from_who_am_i(1140, "fw1.3-harp1.13"))

In [20]:
file.read()

'%YAML 1.1\n---\n# yaml-language-server: $schema=https://raw.githubusercontent.com/harp-tech/reflex-generator/main/schema/device.json\ndevice: Olfactometer\nwhoAmI: 1140\nfirmwareVersion: "1.3"\nhardwareTargets: "1.0"\nregisters:\n  EnableFlow:\n    address: 32\n    access: Write\n    type: U8\n    maskType: EnableFlag\n    description: Starts or stops the flow in all channels.\n  Flowmeter:\n    address: 33\n    access: Event\n    length: 5\n    type: S16\n    description: Value of single ADC read from all flowmeter channels.\n    payloadSpec:\n      Channel0:\n        offset: 0\n      Channel1:\n        offset: 1\n      Channel2:\n        offset: 2\n      Channel3:\n        offset: 3\n      Channel4:\n        offset: 4\n  DI0State:\n    address: 34\n    access: Event\n    type: U8\n    maskType: DigitalState\n    description: State of the digital input pin 0.\n  Channel0UserCalibration: &channel0UserCalibration\n    address: 35\n    access: Write\n    length: 11\n    type: U16\n    d